*   Given data or read from file (line1: deg, line2: kJ/mol)

In [1]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.integrate import quad, cumulative_trapezoid
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
import scipy.constants as sc
from pathlib import Path
from numpy.linalg import eigvals

temperature = 700  # K

dihedralA_kcal_mol = np.array([[180, 0.551967], [170, 0.557568],
                               [160, 0.57842], [150, 0.743988], [140, 1.15127],
                               [130, 1.76722], [120, 2.48711], [110, 3.17616],
                               [100, 3.6976], [90, 3.93171], [80, 3.8368],
                               [70, 3.38603], [60, 2.66725], [50, 1.79179],
                               [40, 0.954215], [30, 0.334189], [20, 0.0295074],
                               [10, 0], [0, 0.00945115]])
dihedralB_kcal_mol = np.array([[180, 0.496007], [170, 0.338429],
                               [160, 0.0933325], [150, 0], [140, 0.17495],
                               [130, 0.647714], [120, 1.29963], [110, 1.9583],
                               [100, 2.45073], [90, 2.6726], [80, 2.55251],
                               [70, 2.1469], [60, 1.56547], [50, 0.979727],
                               [40, 0.572042], [30, 0.409372], [20, 0.469453],
                               [10, 0.615015], [0, 0.652945]])

# Convert energy from kcal/mol to kJ/mol
conversion_factor = sc.calorie
dihedralA = np.copy(dihedralA_kcal_mol)
dihedralA[:, 1] *= conversion_factor
dihedralB = np.copy(dihedralB_kcal_mol)
dihedralB[:, 1] *= conversion_factor

# Create symmetric dihedral angle data
newa = np.copy(dihedralA)
newa[:, 0] = -newa[:, 0]  + 360
newb = np.copy(dihedralB)
newb[:, 0] = -newb[:, 0] + 360
dihedralA = np.unique(np.concatenate((dihedralA, newa), axis=0), axis=0)
dihedralB = np.unique(np.concatenate((dihedralB, newb), axis=0), axis=0)

Angle = np.deg2rad(np.array([-11, -11,14,14,-1.4, -1.4, 14,14]))
rotation = np.array([0, 2, 0, 1, 0, 1, 0, 2])
labels = {
    1: {'label': 'dihedralA', 'color': 'b'},
    2: {'label': 'dihedralB', 'color': 'm'},
    # 3: {'label': 'T-E', 'color': 'c'},
    # 4: {"label": "FT-FT", "color": 'g'},
    # 5: {"label": "ADTDI-FT", "color": 'r'},
}

In [2]:
def read_data(file_name):
    '''
    Read data from a file. 
    column 1: angle in degrees 0-180 or 0-360, column 2: potential in kJ/mol, separated by space
    '''
    data = np.loadtxt(file_name)
    data = np.reshape(data, (-1, 2))
    if data[:, 0].max() - data[:, 0].min() != 360:
        mirrored = np.column_stack((-data[:, 0] + 360, data[:, 1]))
        combined = np.vstack((data, mirrored))
        combined = np.unique(combined, axis=0)
        return combined[np.argsort(combined[:,
                                            0])].T  # transpose to get (deg, y)
    else:
        return data[np.argsort(data[:, 0])].T


def fit_function(x, y, fit_type='intepolation', deg=5):

    if fit_type == 'intepolation':
        fitf = interp1d(x, y, kind='cubic', fill_value="extrapolate")
    else:  # cosine
        p = np.polynomial.polynomial.polyfit(np.cos(np.deg2rad(x)), y, deg)

        def fitf(z):
            return np.polynomial.polynomial.polyval(np.cos(np.deg2rad(z)), p)

    return fitf


def make_Mmat(all_data, Angle_rad, rotation_types, temperature):
    kTval = sc.R * temperature / 1000  # in kJ/mol
    M = len(rotation_types)
    A_list = []
    for i in range(M):
        rot_id = int(rotation_types[i])
        theta = float(Angle_rad[i])
        if rot_id == 0:
            m_i, s_i = 1.0, 0.0
        else:
            fitf = all_data[rot_id]
            Z, _ = quad(lambda phi_deg: np.exp(-fitf(phi_deg) / kTval),
                        0,
                        360,
                        limit=1000)
            m_i, _ = quad(lambda phi_deg: np.cos(np.deg2rad(phi_deg)) * np.exp(
                -fitf(phi_deg) / kTval),
                          0,
                          360,
                          limit=1000)
            m_i /= Z
            s_i, _ = quad(lambda phi_deg: np.sin(np.deg2rad(phi_deg)) * np.exp(
                -fitf(phi_deg) / kTval),
                          0,
                          360,
                          limit=1000)
            s_i /= Z

        S = np.array([[m_i, -s_i, 0.0], [s_i, m_i, 0.0], [0.0, 0.0, 1.0]])
        c = np.cos(theta)
        s = np.sin(theta)
        R_y = np.array([[c, 0.0, s], [0.0, 1.0, 0.0], [-s, 0.0, c]])
        A_list.append(S @ R_y)

    # Multiply all A_i for the repeat unit
    Mmat = np.eye(3)
    for A in A_list:
        Mmat = A @ Mmat
    return Mmat


def compute_correlation(Mmat):
    """
    Calculates the correlation length
    """
    # eigen values
    eigs = eigvals(Mmat)
    lambda_max = float(np.max(np.abs(eigs)))

    # if lambda_max is close to 1, numerical stability may be an issue; if >=1 (numerical error), clip to 1 - eps
    if lambda_max >= 1.0:
        eps = 1e-12
        if lambda_max > 1.0 + 1e-8:
            print("Warning: lambda_max > 1 (numerical error)")
        lambda_max = min(lambda_max, 1.0 - eps)

    # 
    corr_length = -1.0 / np.log(lambda_max)
    return corr_length, lambda_max

In [3]:
all_data = {}

for key, data_label in labels.items():
    if key == 1:
        degree, energy = dihedralA.T
        # all_data[key] = fit_function(degree, energy, fit_type='cosine', deg=5)
    elif key == 2:
        degree, energy = dihedralB.T
    all_data[key] = fit_function(degree, energy, fit_type='cosine', deg=5)

matrix = make_Mmat(all_data, Angle, rotation, temperature)

In [4]:
lp_repeats, lam = compute_correlation(matrix)

print(f"Max eigen value: lambda_max = {lam:.12f}")
print(f"Correlation length = {lp_repeats:.6f}")

Max eigen value: lambda_max = 0.692924185631
Correlation length = 2.726024
